In [4]:
import cantera as ct
import numpy as np
import scipy as sp
import pandas as pd
from pint import UnitRegistry
import plotly.express as px
# import pypropep as ppp
import yaml
import time
import CEA_Wrap as CEA

ureg = UnitRegistry()
Q = ureg.Quantity

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 12)


In [39]:
class EngineState:
    '''
    This class will define the properties of an rocket engine at the chamber, throat and exit 
    given the propellant, and conditions that engine is in.

    Parameters
    --------------------------------------------------
    oxidizer : string
        A string of a species name which is used to define a cantera solution representing the oxidizer.
        Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
    fuel : string
        Similar to oxidizer, this is a string of a species name which is used to define a cantera solution representing the fuel.
        Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
    of_ratio : float
        the mass ratio of oxidizer to fuel defined as mass_oxidizer / mass_fuel.
    pressure : float
        The pressure of the combustion chamber in units of Pascals (Pa).
    exit_value : float
        The exit conditions of the engine. Depending on what 'exit parameter' is
        defined to be the exit condition can be defined in the following ways:
            'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
            'area ratio': Defined as area_exit / area_throat should be value greater than 1 TODO: add functionality
    exit_parameter : string
        A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
        or 'area ratio'. 
    temp_oxidizer : float
        A positive float value defining the temperature of the oxidizer in Kelvin (K). If the oxidizer is liquid the temperature 
        is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
    temp_fuel : float
        Similar to above. positive float value defining the temperature of the fuel in Kelvin (K). If the fuel is liquid the temperature 
        is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
    transport_species Optional: [list | None]
        A list of species 

        

    TODO board: 
    --------------------------------------------------
        TODO: apply efficiency.
        TODO: apply flow seperation
        TODO: improve chemical database
        TODO: allow fluid temp setting
        TODO: condensed species capabilities
        TODO: Improve convergence *especially for locations along the length of the chamber*
        TODO: Ambient pressure
        TODO: Ambient pressure Array
        TODO: add Pint https://pint.readthedocs.io/en/stable/advanced/wrapping.html#wrapping
        TODO: add FAC capabilities
        TODO: add frozen

    Attributes:
    --------------------------------------------------
    Properties : Pandas Dataframe
    
    Methods
    --------------------------------------------------

    A class for the state of a rocket engine. Given the propellants and conditions of a rocket engine this 
    class will define the properties of an engine at
    '''

    def __init__(self, oxidizer, fuel, of_ratio, pressure, exit_value, gas, condensate = None, size_value=None, size_parameter ="thrust",  exit_parameter="pressure", transport=None):
        '''
        Initializes instance of class. See class description for details. 
        '''

        # initializes propellants
        self.of_ratio = of_ratio
        self.pressure = pressure
        start_time = time.time()
        oxidizer.TP = oxidizer.T, pressure
        oxidizer.equilibrate('TP')
        self.oxidizer = oxidizer
        fuel.TP = fuel.T, pressure
        fuel.equilibrate('TP')
        self.fuel = fuel
        self.gas = gas
        self.condensate = condensate
    
        chem_init_time = time.time()- start_time
        self.__chamber_properties(self.gas, self.condensate)
        chamber_time = time.time()- start_time - chem_init_time
        self.__throat_properties(self.gas, self.condensate)
        throat_time = time.time()- start_time - chamber_time -chem_init_time
        self.__exit_properties(self.gas, self.condensate, exit_value, exit_parameter=exit_parameter)
        exit_time = time.time()- start_time - throat_time - chamber_time -chem_init_time

        if  size_value != None:
            self.size_engine(size_value, size_parameter=size_parameter)
        size_time = time.time()- start_time - exit_time - throat_time - chamber_time -chem_init_time


        # self.engine_state_dict = {'chamber' : self.chamber, 'throat' : self.throat, 'exit' : self.exit }
        self.engine_state = pd.DataFrame([self.chamber, self.throat, self.exit], index=["chamber", "throat", "exit"])

        df_time = time.time()- start_time-size_time - exit_time - throat_time - chamber_time -chem_init_time

        print(f"-----\ntotal time: {time.time()-start_time}\nchemistry initialization: {chem_init_time}\nchamber properties: {chamber_time}\nthroat time: {throat_time}\nexit time: {exit_time}\nsizing time: {size_time}\ndf making: {df_time} ")

    def __call__(self):
        return self.engine_state
    
    def __str__(self):
        return self.engine_state.to_string()
        
    def __get_thermo_derivatives(self, gas, condensate):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species this method will find thermodynamic derivatives.
        These derivatives can then be used to calculate thermodynamic properties, or create interpolation/extrapolation of engine properties 

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.
        The equations used are 2.50, 2.51, 2.56, 2.57, 2.58, 2.64, 2.65, 2.66 

        TODO: add condensed species to implementation
        TODO: explore interpolation of engine properties using derivatives
        
        Parameters
        --------------------------------------------------
        mixture : Cantera Mixture
            A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        
        Returns
        --------------------------------------------------
        Derivatives : dictionary:
            A dictionary consisting of the derivatives this function calculates. The keys and descritions of said dictionaries are as follows
        
            dpi_dlnT_P : list
                A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
                and lambda is the langrangian multiplier
            dlnn_dlnT_P : float
                The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas
            dpi_dlnP_T : list
                A list of the derivative of pi with respect to ln(P) at constant temperature for each element.
            dlnn_dlnP_T : float
                The derivative of ln(n) with respect to ln(P) at constant tempearture.
            dlnV_dlnT_P : float
                The derivative of ln(V) with respect to ln(T) at constant pressure.
            dlnV_dlnP_T : float
                The derivative of ln(V) with respect to ln(P) at constant tempearture.
        '''
                    
        # Defines the number of moles of each species in the mixture

        # if not condensate:
        moles = gas.X * (1/ gas.mean_molecular_weight)
        n_elements = gas.n_elements
        num_variables = 2 * gas.n_elements + 2
        # else: 
        #     moles = gas.X * (1/ gas.mean_molecular_weight) + condensate.X * (1/condensate.mean_molecular_weight)
        #     n_elements = len(set(gas.element_names + condensate.element_names))
        #     num_variables = 2 * gas.n_elements + 2 + 2 * 

        # Initializing Solution Matrices table 2.3 and 2.4 in RP-1311
        coeff_matrix = np.zeros((num_variables, num_variables))
        right_hand_side = np.zeros(num_variables)

        # Initializes a_ij
        a_ij = np.zeros((gas.n_elements, gas.n_species))
        for i, element in enumerate(gas.element_names): 
            for j, species in enumerate(gas.species_names):
                a_ij[i,j] = gas.n_atoms(species, element)


        # Coefficients for equation 2.56 TODO: add terms for condensed species
        for k in range(gas.n_elements):
            for i in range(gas.n_elements):
                coeff_matrix[k,i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
            coeff_matrix[k, gas.n_elements] = np.sum(a_ij[k,:] * moles)
            right_hand_side[k] = -np.sum(a_ij[k,:] * moles * gas.standard_enthalpies_RT)

        # TODO add equation 2.57 (it is for condensed species)
        # for i in range():
        #     None


        # Coefficients for equation 2.58 
        for i in range(gas.n_elements):
            coeff_matrix[gas.n_elements, i] = np.sum(a_ij[i, :] * moles)
        right_hand_side[gas.n_elements] = -np.sum(moles * gas.standard_enthalpies_RT)

        # Coefficients for equation 2.64 TODO: add terms for condensed species
        for k in range(gas.n_elements):
            for i in range(gas.n_elements):
                coeff_matrix[gas.n_elements+1+k,gas.n_elements+1+i] = np.sum(a_ij[k,:] * a_ij[i,:] * moles)
            coeff_matrix[gas.n_elements+1+k, 2*gas.n_elements+1] = np.sum(a_ij[k,:] * moles)
            right_hand_side[gas.n_elements+1+k] = np.sum(a_ij[k,:] * moles)

        # TODO add equation 2.65 (it is for condensed species)
        

        # Coefficeints for equation 2.66
        for i in range(gas.n_elements):
            coeff_matrix[2*gas.n_elements+1, gas.n_elements+1+i] = np.sum(a_ij[i, :] * moles)
        right_hand_side[2*gas.n_elements+1] = np.sum(moles)

        # Solve for the derivatives define them based off table 2.3, 2.4 and equation 2.50 and 2.51 
        derivs = np.linalg.solve(coeff_matrix, right_hand_side)
        derivatives = { "dpi_dlnT_P"    : derivs[0 : gas.n_elements], 
                        "dlnn_dlnT_P"   : derivs[gas.n_elements], 
                        "dpi_dlnP_T"    : derivs[gas.n_elements + 1: 2 * gas.n_elements + 1],
                        "dlnn_dlnP_T"   : derivs[2 * gas.n_elements + 1], 
                        "dlnV_dlnT_P"   : 1 + derivs[gas.n_elements], 
                        "dlnV_dlnP_T"   : -1 + derivs[2 * gas.n_elements + 1]}

        return derivatives

    def __get_thermo_properties(self, gas, condensate, dpi_dlnT_P, dlnn_dlnT_P, dlnV_dlnT_P, dlnV_dlnP_T):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species, as well as certain thermdynamic derivatives of said mixture, this function will find 
        the thermodynamic properties of the mixture

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        TODO: add condensed species to implementation
        TODO: add capability to solve for thermal conductivity, viscocity, and prandtl number  

        Parameters
        --------------------------------------------------
        mixture : Cantera Mixture
            A solution or mixture of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        dpi_dlnT_P : list
            A list of the derivative of pi with respect to ln(T) at constant pressure for each element, where pi is -lambda/RT, 
            and lambda is the langrangian multiplier
        dlnn_dlnT_P : float
            The derivative of ln(n) with respect to ln(T) at constant pressure. Where n is the number of moles of gas.
        dlnV_dlnT_P : float
            The derivative of ln(V) with respect to ln(T) at constant pressure.
        dlnV_dlnP_T : float
            The derivative of ln(V) with respect to ln(P) at constant tempearture.
            
        Returns
        --------------------------------------------------
        properties : dictionaries
            A dictionary consisting of the properties this function calculates. The keys and descritions of said dictionaries are as follows

            Pressure : float
            Temperature : float
            density : float
            specific_volume : float
            enthalpy : float
            internal_energy : float
            gibbs : float
            entropy : float
            molar_mass : float
            c_p : float
                the specific heat at constant pressure
            c_v : float
                the specific heat at constant volume
            gamma : float
                the specific heat ratio
            gamma_s : float
                defined as derivative ln(P) with respect to ln(rho) at constant entropy per equation 2.71 in RP-1311
            speed_sound : float
                the speed of sound in the mixture
        '''

        # Defines the number of moles of each species in the micture
        moles = gas.X * (1/ gas.mean_molecular_weight)

        # Initializes a_ij
        a_ij = np.zeros((gas.n_elements, gas.n_species))
        for i, element in enumerate(gas.element_names): 
            for j, species in enumerate(gas.species_names):
                a_ij[i,j] = gas.n_atoms(species, element)

        # Finds specific heat at constant pressure based on equation 2.59 TODO: add terms for condensed species
        c_p =  ct.gas_constant * (
            np.sum([dpi_dlnT_P[i] * np.sum(a_ij[i,:] * moles * gas.standard_enthalpies_RT) for i in range(gas.n_elements)]) +
            np.sum(moles * gas.standard_enthalpies_RT) * dlnn_dlnT_P +
            np.sum(moles * gas.standard_cp_R) +
            np.sum(moles * gas.standard_enthalpies_RT**2)
        )

        # Finds specifc heat at constant volume, based on equation 2.70, specific heat ratio, The isentropic exponent based on equation 2.73, 
        # and speed of sound based on equation 2.74
        c_v = c_p + gas.P * gas.v / gas.T * dlnV_dlnT_P**2 / dlnV_dlnP_T
        gamma = c_p / c_v
        gamma_s = -gamma/dlnV_dlnP_T
        speed_sound = np.sqrt(ct.gas_constant * gas.T * gamma_s/gas.mean_molecular_weight)

        properties = {  "of ratio"          : self.of_ratio,
                        "pressure"          : gas.P,
                        "temperature"       : gas.T,
                        "density"           : gas.density_mass,
                        "specific volume"   : gas.volume_mass,
                        "enthalpy"          : gas.enthalpy_mass,
                        "internal energy"   : gas.int_energy_mass,
                        "gibbs"             : gas.gibbs_mass,
                        "entropy"           : gas.entropy_mass,
                        "molar mass"        : gas.mean_molecular_weight,
                        "c_p"               : c_p,
                        "c_v"               : c_v,
                        "gamma"             : gamma,                                                                            
                        "gamma_s"           : gamma_s,    
                        "speed sound"       : speed_sound          
                        }

        return properties
    
    def __get_transport_properties(self, gas, transport):
        '''
        TODO: Update  comment
        '''

        return

    def __chamber_properties(self, gas, condensate):
        '''
        This is an internal method not meant for use outside the class.
        given a mixture of gases & condensed species, as well as certain thermdynamic properties of said mixture, this function will find 
        properties of the engine

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        Parameters
        --------------------------------------------------
        products : Cantera Mixture 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
            
        Returns
        --------------------------------------------------
        properties : dictionary
        '''

        # Equilibriates chamber returning a cantera mixture with the properties & composition at the chamber
        molar_ratio = self.of_ratio / (self.oxidizer.mean_molecular_weight / self.fuel.mean_molecular_weight)
        moles_ox = molar_ratio / (1 + molar_ratio)
        moles_f = 1 - moles_ox

        chamber_mixture = ct.Mixture([(self.fuel, moles_f), (self.oxidizer, moles_ox), (gas, 0)])
        chamber_mixture.equilibrate('HP', log_level=1, estimate_equil=-1)

        # Finds thermodynamic derivatives 
        derivatives = self.__get_thermo_derivatives(gas, condensate)

        # Finds thermodynamic properties
        therm_prop = self.__get_thermo_properties(gas, condensate, derivatives["dpi_dlnT_P"], derivatives["dlnn_dlnT_P"], derivatives["dlnV_dlnT_P"], derivatives["dlnV_dlnP_T"])

        # Calculate c* per
        char_velocity = (np.sqrt(ct.gas_constant * therm_prop["temperature"] / (therm_prop["molar mass"] * therm_prop["gamma"])) * 
                            np.power(2 / (therm_prop["gamma"] + 1), -(therm_prop["gamma"] + 1) / (2*(therm_prop["gamma"] - 1))))

        # velocity and Mach are 0 in a FAC combustor, area ratio, Isp, Ivac, and Cf are not defined at chamber
        chamber_prop = {"velocity"        : 0, 
                        "mach"            : 0, 
                        "area ratio"      : np.nan,
                        "I_sp"            : np.nan,
                        "I_vac"           : np.nan,
                        "c*"              : char_velocity,
                        "C_f"             : np.nan,
                        "mole fraction"   : {k:v for k,v in gas.mole_fraction_dict().items() if v > 10e-6}}
        
        self.chamber = therm_prop | chamber_prop | derivatives

    def __throat_properties(self, gas, condensate):
        '''
        Description
        --------------------------------------------------
        This is an internal method not meant for use outside the class.
        given a solution of gases & condensed species, this function will find 
        properties of the engine

        The theory used for these calculations can be found in section 2.5 and 2.6 of RP-1311.

        Parameters
        --------------------------------------------------
        products : Cantera Solution 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
            
        Returns
        --------------------------------------------------
        None
        '''

        chamber = self.chamber
        # initial guess at throat pressure using specific heat ratio gamma
        pressure_throat = chamber["pressure"] / np.power((chamber["gamma_s"] + 1) / 2., chamber["gamma_s"] / (chamber["gamma_s"] - 1))

        # Setting up for iteration
        max_iter_throat = 10            # NOTE exceeds value of 4 from RP-1311
        tolerance_throat = 4e-5
        mach = 1.0
        num_iter = 0
        residual = 1
 
        while (residual > tolerance_throat and num_iter < max_iter_throat ) :
            num_iter += 1
           
            gas.SP = chamber["entropy"], pressure_throat
            gas.equilibrate('SP')

            throat_derivatives = self.__get_thermo_derivatives(gas, condensate)
            throat_properties = self.__get_thermo_properties(gas, condensate, throat_derivatives["dpi_dlnT_P"], throat_derivatives["dlnn_dlnT_P"], throat_derivatives["dlnV_dlnT_P"], throat_derivatives["dlnV_dlnP_T"])
            
            velocity = np.sqrt(2 * (chamber["enthalpy"] - throat_properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * throat_properties["temperature"] * throat_properties["gamma_s"]  / throat_properties["molar mass"])
            mach = velocity / speed_sound
            
            pressure_throat = pressure_throat * (1 + throat_properties["gamma_s"] * mach**2) / (1 + throat_properties["gamma_s"] )

            residual = np.abs((velocity**2 - speed_sound**2)/velocity**2)

            # print(f"pressure is: {pressure_throat:.7}\t velocity is {velocity:.4}\t speed of sound is: {speed_sound:.4}\t residual is: {np.abs((velocity**2 - speed_sound**2)/velocity**2):.4}")

        if num_iter >= max_iter_throat:
            print(f'Warning: Convergance took {num_iter} iterations which exceeds the limit of {max_iter_throat} max iterations. residual is {residual} which exceeds tolerance of {tolerance_throat}.')
        
        # velocity and Mach are the same at throat, area ratio, Isp, Ivac, and Cf are not defined at chamber
        throat_prop = { "velocity"        : speed_sound, 
                        "mach"            : 1, 
                        "area ratio"      : np.nan,
                        "I_sp"            : np.nan,
                        "I_vac"           : np.nan,
                        "c*"              : np.nan,
                        "C_f"             : np.nan,
                        "mole fraction"   :  {k:v for k,v in gas.mole_fraction_dict().items() if v > 10e-6}}

        self.throat = throat_properties | throat_prop | throat_derivatives

    def __exit_properties(self, gas, condensate, exit_value, exit_parameter='pressure'):
        '''
        This is an internal method not meant for use outside the class.
        equilbriates solution and finds thermal derivatives and properties at the exit of the engine.
      
        Parameters
        --------------------------------------------------
        products : Cantera Solution 
            A solution of primarily gases (ideal gas assumptions will hold up to several percent condensed species by mass)
        exit_value : float
            The exit conditions of the engine. Depending on what 'exit parameter' is
            defined to be the exit condition can be defined in the following ways:
                'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
                'area ratio': Definfed as area_exit / area_throat should be value greater than 1 TODO: add functionality
        exit_parameter : string
            A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
            or 'area ratio'. 
            
        Returns
        --------------------------------------------------
        None
        '''

        if exit_parameter == "area ratio":
            exit_properties = self.state_at_area(exit_value, speed = "supersonic")
            ae_mdot = 1/(exit_properties["density"]*exit_properties["velocity"])

            exit_properties["I_sp"] = exit_properties["velocity"] / sp.constants.g
            exit_properties["I_vac"] = exit_properties["velocity"] / sp.constants.g + exit_properties["pressure"] * ae_mdot / sp.constants.g
            exit_properties["C_f"] = exit_properties["velocity"] / self.chamber["c*"]

            self.exit =  exit_properties
        
        elif exit_parameter == "pressure":
            pressure = exit_value
            gas.SP = self.chamber["entropy"], pressure 
            gas.equilibrate('SP')

            exit_derivatives = self.__get_thermo_derivatives(gas, condensate)
            exit_properties = self.__get_thermo_properties(gas, condensate, exit_derivatives["dpi_dlnT_P"], exit_derivatives["dlnn_dlnT_P"], exit_derivatives["dlnV_dlnT_P"], exit_derivatives["dlnV_dlnP_T"])

            velocity = np.sqrt(2* (self.chamber["enthalpy"] - exit_properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * exit_properties["temperature"] * exit_properties["gamma_s"]  / exit_properties["molar mass"])
            mach = velocity / speed_sound

            at_mdot = 1 / (self.throat["density"]*self.throat["velocity"])
            ae_mdot = 1 / (exit_properties["density"]*velocity)
            ae_at = ae_mdot/at_mdot

            # print(pressure, ae_mdot, velocity, )
            Isp = velocity / sp.constants.g
            Ivac = Isp + pressure * ae_mdot / sp.constants.g
            Cf = velocity / self.chamber["c*"]

            exit_prop = {   "velocity"        : velocity, 
                            "mach"            : velocity/speed_sound, 
                            "area ratio"      : ae_at,
                            "I_sp"            : Isp,
                            "I_vac"           : Ivac,
                            "c*"              : np.nan,
                            "C_f"             : Cf,
                            "mole fraction"   :  {k:v for k,v in gas.mole_fraction_dict().items() if v > 10e-6}}

            exit_properties = exit_properties | exit_prop | exit_derivatives

            self.exit = exit_properties
            
        else: 
            raise ValueError("Invalid input. exit_parameter was not defined as 'pressure' or 'area ratio'")

    def size_engine(self, size_value, size_parameter = 'thrust'):
        '''
        Description
        --------------------------------------------------
        Given thrust, throat diameter, or massflow will define the other three and update engine_state accordingly

        Parameters
        --------------------------------------------------
        size_value : float
            Depending on what 'size_parameter' is defined in the following ways
                'thrust' : the target thrust of the engine in units newtons
                'mass flow' : the target mass flow of the engine in units kilogram/second
                'throat diameter' : the diameter of the throat in units meter
        size_parameter : string
            a string with three possible values:
                'thrust' 
                'mass flow'
                'throat diameter' 

        Returns
        --------------------------------------------------
        None
        '''

        gamma = self.throat["gamma"]
        if size_parameter == 'thrust': 
            thrust = size_value
            mass_flow = thrust / self.exit['velocity']
            throat_area = mass_flow / (self.throat['velocity'] * self.throat['density'])
            throat_area_2 = np.sqrt(self.chamber['temperature'])/self.chamber['pressure'] * 1/(np.sqrt(self.throat['gamma']/(ct.gas_constant/self.throat['molar mass']))*
                            ((self.throat['gamma']+1)/2)**((self.throat["gamma"]+1)/(2*self.throat["gamma"]-2)))
            throat_area_3 = thrust / (self.exit['C_f'] * self.chamber['pressure'])
            throat_diameter = np.sqrt(throat_area/np.pi) / 2
            throat_diameter_2 = np.sqrt(throat_area_2/np.pi) / 2
            throat_diameter_3 = np.sqrt(throat_area_3/np.pi) / 2


        elif size_parameter == 'mass flow':
            mass_flow = size_value
            thrust = mass_flow * self.exit['velocity']
            throat_area = mass_flow / (self.throat['velocity'] * self.throat['density'])
            throat_diameter = np.sqrt(throat_area/np.pi) / 2
        elif size_parameter == 'throat diameter':
            throat_diameter = size_value
            throat_area = np.pi * (throat_diameter/2)**2
            mass_flow = throat_area * self.throat['velocity'] * self.throat['density']
            thrust = mass_flow * self.exit['velocity']
        else:
            raise ValueError("Invalid input. size_parameter was not defined as 'thrust', 'mass flow' or 'throat diameter'")
        
        chamber_size = {'thrust' :      np.nan,
                        'mass flow' :   mass_flow,
                        'area' :        np.inf,
                        'diameter' :    np.inf}
        
        throat_size =  {'thrust' :      np.nan,
                        'mass flow' :   mass_flow,
                        'area' :        throat_area,
                        'area 2' :        throat_area_2,
                        'area 3' :        throat_area_3,
                        'diameter' :    throat_diameter,
                        'diameter 2' :    throat_diameter_2,
                        'diameter 3' :    throat_diameter_3,}

        exit_size =    {'thrust' :      thrust,
                        'mass flow' :   mass_flow,
                        'area' :        throat_area * self.exit['area ratio'],
                        'diameter' :    np.sqrt(throat_area * self.exit['area ratio']/np.pi) / 2}
        
        self.chamber.update(chamber_size)
        self.throat.update(throat_size)
        self.exit.update(exit_size)

        self.engine_state = pd.DataFrame([self.chamber, self.throat, self.exit], index=["chamber", "throat", "exit"])
            
    def state_at_area(self, area_ratio, speed = "supersonic"):
        '''
        Description
        --------------------------------------------------
        Iteratively find the properties at a given location in along the nozzle given the area ratio and whether it is in the subsonic or supersonic section of the nozzle.

        Parameters
        --------------------------------------------------
        area_ratio : float
            The ratio at of the area at a given location over the area of the throat.
        speed : string
            a string with two possible values:
                'subsonic' : the converging section of the nozzle
                'supersonic' : the diverging section of the nozzle

        Returns
        --------------------------------------------------
        local_properties : Dict
            A dictionary of the thermodynamic properties and derivatives 
        
        '''

        # checking for valid input
        if area_ratio <= 1:
            raise ValueError("Area ratio was less than or equal to 1")
        
        # getting initial guess of ln(pc/pe).
        if speed == "subsonic": 
            if area_ratio > 1.000 and area_ratio < 1.09:
                lnpc_p = 0.9*np.log(self.chamber["pressure"]/self.throat["pressure"])/(area_ratio+10.587*np.log(area_ratio)**3+9.454*np.log(area_ratio))
            elif area_ratio >= 1.09:
                lnpc_p = np.log(self.chamber["pressure"]/self.throat["pressure"])/(area_ratio+10.587*np.log(area_ratio)**3+9.454*np.log(area_ratio)) 
                
        if speed == "supersonic": 
            if area_ratio > 1.000 and area_ratio < 2:
                lnpc_p = np.log(self.chamber["pressure"]/self.throat["pressure"]) +np.sqrt(3.294*np.log(area_ratio)**2+1.534*np.log(area_ratio))
            elif area_ratio >= 2:
                lnpc_p = self.chamber["gamma_s"] + 1.4 * np.log(area_ratio)
        
        # initial guess at equilibrium
        pressure = self.chamber["pressure"]/np.exp(lnpc_p)
        products = self.gas
        products.SPX = self.throat["entropy"], pressure, self.throat["mole fraction"]
        products.equilibrate("SP")

        # defining iteration limits and convergence
        num_iter = 0
        max_iter = 10
        tolerance = 4e-5
        residual = 1

        # defines throat area / throat massflow
        at_mdot = 1 / (self.throat["density"]*self.throat["velocity"])

        # iterative solver for state at position _blank_
        while residual > tolerance:
            num_iter += 1

            if num_iter == max_iter:
                print(f"exceeded {max_iter} iterations, residual is {residual} which is not below tolerance of {tolerance}")
                break

            derivatives = self.__get_thermo_derivatives(products, self.condensate)
            properties = self.__get_thermo_properties(products, self.condensate, derivatives["dpi_dlnT_P"], derivatives["dlnn_dlnT_P"], derivatives["dlnV_dlnT_P"], derivatives["dlnV_dlnP_T"])

            velocity = np.sqrt(2 * (self.chamber["enthalpy"] - properties["enthalpy"]))
            speed_sound = np.sqrt(ct.gas_constant * properties["temperature"] * properties["gamma_s"] / properties["molar mass"])
            a_mdot = 1/(properties["density"]*velocity)
            a_at = a_mdot/at_mdot

            dlnpc_p_dlna_at = properties["gamma_s"] * velocity**2 / (velocity**2 - speed_sound**2)
            lnpc_p = lnpc_p + dlnpc_p_dlna_at * (np.log(area_ratio) - np.log(a_at))
            residual = abs(dlnpc_p_dlna_at * (np.log(area_ratio) - np.log(a_at)))

            # print(f"residual: {residual} \t\t pressure: {pressure}\t\t speed of sound: {speed_sound}")

            pressure = self.chamber["pressure"]/np.exp(lnpc_p)

            products.SP = self.throat["entropy"], pressure
            products.equilibrate('SP')

        
        local_prop  = { "velocity"        : velocity, 
                        "mach"            : velocity/speed_sound, 
                        "area ratio"      : area_ratio,
                        "I_sp"            : np.nan,
                        "I_vac"           : np.nan,
                        "c*"              : np.nan,
                        "C_f"             : np.nan,
                        "mole fraction"   : products.mole_fraction_dict()}

        local_proprties = properties | local_prop | derivatives

        return local_proprties
    
    def property(self, location, variable):
        if location in self.engine_state.index and variable in self.engine_state.columns:
            return self.engine_state[variable][location]

In [40]:
class Engine(EngineState):     
    def __init__(self,  oxidizer, fuel, of_ratio, pressure, exit_value, gas,  size_value, condensate = None, size_parameter ="thrust",
                 temp_oxidizer=None, temp_fuel=None, combustion_products=None, exit_parameter="pressure"):
        '''
        This class will define the properties of an rocket engine at the chamber, throat and exit 
        given the propellant, and conditions that engine is in.

        Parameters
        --------------------------------------------------
        oxidizer : string
            A string of a species name which is used to define a cantera solution representing the oxidizer.
            Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
        fuel : string
            Similar to oxidizer, this is a string of a species name which is used to define a cantera solution representing the fuel.
            Define based off the list of phase names from the propellants.yaml file TODO: kill propellants.yaml and create a list users can choose from 
        of_ratio : float
            the mass ratio of oxidizer to fuel defined as mass_oxidizer / mass_fuel.
        pressure : float
            The pressure of the combustion chamber in units of Pascals (Pa).
        exit_value : float
            The exit conditions of the engine. Depending on what 'exit parameter' is
            defined to be the exit condition can be defined in the following ways:
                'pressure' (default): Exit pressure of the gases in Pa should be greater than 0 but lower than chamber pressure
                'area ratio': Definfed as area_exit / area_throat should be value greater than 1 TODO: add functionality
        size_value : float 
            Determines the size of the engine. depending on what 'size parameter' is defined to be. 
            exit condition can be defined in the following way
        exit_parameter : string
            A string which determines how you are defining the exit condition of your engine. Can either be set to 'pressure' (default)
            or 'area ratio'. 
        temp_oxidizer : float
            A positive float value defining the temperature of the oxidizer in Kelvin (K). If the oxidizer is liquid the temperature 
            is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
        temp_fuel : float
            Similar to above. positive float value defining the temperature of the fuel in Kelvin (K). If the fuel is liquid the temperature 
            is assumed to be saturated temperatures at standard pressure, and gaseous oxidizer will be set to a room tempearture of 295.15K. TODO: add functionality
        

        Attributes:
        --------------------------------------------------
        Properties : Pandas Dataframe
        
        Methods
        --------------------------------------------------

        A class for the state of a rocket engine. Given the propellants and conditions of a rocket engine this 
        class will define the properties of an engine at
        '''
        super().__init__(oxidizer, fuel, of_ratio, pressure, exit_value, gas, size_value = size_value, condensate=condensate, size_parameter= size_parameter,
                        exit_parameter=exit_parameter)
        
    def _chamber_contour(self, length_value, contraction_ratio, contraction_angle=30, nozzle_inlet_radius_ratio=0.5, 
                         throat_inlet_radius_ratio = 1.5, length_parameter = 'characteristic length'):
        '''
        Description
        --------------------------------------------------
        will create

        Parameters
        --------------------------------------------------
        leng_value : float
            Depending on what 'size_parameter' is defined in the following ways
                'thrust' : the target thrust of the engine in units newtons
                'mass flow' : the target mass flow of the engine in units kilogram/second
                'throat diameter' : the diameter of the throat in units meter
        length_parameter : string
            a string with three possible values:
                'characteristic length' 
                'chamber length'
                'throat diameter' 

        Returns
        --------------------------------------------------
        None
        '''

        theta_c = np.radians(contraction_angle)

        r_t = self.throat["diameter"]/2
        r_c = np.sqrt(contraction_ratio * r_t**2)
        r_tin = throat_inlet_radius_ratio * r_t
        r_ninmax = (r_c-r_t)/(1-np.cos(theta_c)) - r_tin
        r_nin = r_ninmax * nozzle_inlet_radius_ratio

        
        # checks for valid inputs: 
        if not nozzle_inlet_radius_ratio <=1:
            raise ValueError("the nozzle_inlet_radius_ratio must not exceed 1. This is the ratio of the radius/ maximum possible radius")
        if r_tin *(1- np.cos(theta_c)) > r_c-r_t:
            raise ValueError("throat_inlet_radius_ratio is too high. The nozzle geometry is not possible with the given contraction ratio and contraction angle")
        

        # defines bounds for x.        
        x_tin = -r_tin * np.sin(theta_c)
        x_nin = -1/np.tan(theta_c)*(r_c-r_nin*(1-np.cos(theta_c))-
                                                    (r_t+r_tin*(1-np.cos(theta_c))))-r_tin*np.sin(theta_c)
        x_c = x_nin - r_nin * np.sin(theta_c)

        # find volume of converging section
        volume_throat_inlet = np.pi*(-x_tin**3/3 + (r_t + r_tin)*r_tin**2*(np.asin(x_tin/r_tin)+np.sin(2*np.asin(x_tin/r_tin))/2))
        sub = r_t+r_tin*(1-np.cos(theta_c)-np.tan(theta_c)*np.sin(theta_c))
        volume_nozzle_line = np.pi*(np.tan(theta_c)**2*x_tin**3/3-np.tan(theta_c)*sub*x_tin**2+sub**2*x_tin) - np.pi*(np.tan(theta_c)**2*x_nin**3/3-np.tan(theta_c)*sub*x_nin**2+sub**2*x_nin) 
        volume_nozzle_inlet =  np.pi*(-x_nin**3/3+x_c*x_nin**2+(r_c-r_nin)*r_nin**2*(np.asin((x_nin-x_c)/r_nin)+np.sin(2*np.asin((x_nin-x_c)/r_nin))/2)+((r_c-r_nin)**2+r_nin**2-x_c**2)*x_nin) -np.pi*(2*x_c**3/3+((r_c-r_nin)**2+r_nin**2-x_c**2)*x_c) 
        volume_converging = volume_throat_inlet + volume_nozzle_line + volume_nozzle_inlet
        

        # find length of chamber
        if length_parameter == 'characteristic length':
            l_star = length_value            
            volume_total = self.throat["area"] * length_value

            if volume_converging > volume_total:
                raise ValueError("the converging section volume is larger than the total volume of your. try increasing your L* or reducing your contraction ratio.")
            volume_chamber = volume_total - volume_converging

            l_c = volume_chamber/(np.pi*r_c**2)
        elif length_parameter == 'chamber length':
            l_c = length_value
            volume_chamber = np.pi * r_c **2 * l_c
            volume_total = volume_chamber + volume_converging
            l_star = volume_total / self.throat["area"]

        x_inj = x_c - l_c

        print(f"r_t: {r_t}, r_c: {r_c}, r_tin: {r_tin}, r_nin = {r_nin}")
        print(f"throat_inlet: {x_tin}, nozzle_inlet: {x_nin}, chamber_end: {x_c}, x_inj: {x_inj}")
        print(f"volume_throat_inlet: {volume_throat_inlet}, volume_nozzle_line: {volume_nozzle_line}, volume_nozzle_inlet: {volume_nozzle_inlet}, volume_converging: {volume_converging}, volume_chamber: {volume_chamber}, volume_total: {volume_total}")

        x_dict = {"x_inj" : x_inj,  "x_c" : x_c, "x_nin" : x_nin, "x_tin" : x_tin, "x_t" : 0}

        def chamber_contour(x): 
            # Defines throat
            
            if x == 0:
                return r_t

            # defines radius before throat
            elif x_tin <= x and x < 0:
                return -np.sqrt(r_tin**2 - x**2) + (r_tin + r_t)
            
            # defines contraction line
            elif x_nin <= x and x < x_tin:
                return -np.tan(theta_c) * (x + r_tin * np.sin(theta_c))+r_t+r_tin*(1-np.cos(theta_c))
            
            # defines radius befor nozzle
            elif x_c <= x and x < x_nin: 
                return np.sqrt(r_nin**2 - (x-x_c)**2) + (r_c - r_nin)

            # defines chamber section
            elif x_inj <= x and x < x_c:
                return r_c
            
            else: 
                raise ValueError(f"x coordinate exceeds bounds of chamber. please make sure {x_inj} <= x <= 0")

        return chamber_contour, x_dict

    def conical_contour(self, length_value, contraction_ratio, contraction_angle=30, nozzle_inlet_radius_ratio=0.5, throat_inlet_radius_ratio = 1.5, 
                                length_parameter = 'characteristic length', throat_outlet_radius_ratio=0.382, expansion_angle=15, fidelity = 500):
        
        chamber_contour, chamber_x_dict = self._chamber_contour(length_value, contraction_ratio, contraction_angle, nozzle_inlet_radius_ratio, throat_inlet_radius_ratio, length_parameter = 'characteristic length')

        theta_e = np.radians(expansion_angle)
        r_t = self.throat["diameter"]/2
        r_tout = throat_outlet_radius_ratio * r_t
        print(r_tout)
        x_tout = r_tout * np.sin(theta_e)
        x_e = (self.exit["diameter"]/2-r_t-r_tout*(1-np.cos(theta_e)))/np.tan(theta_e)+r_tout*np.sin(theta_e)
        
        nozzle_x_dict = {"x_tout" : x_tout, "x_e" : x_e}

        x_dict = chamber_x_dict | nozzle_x_dict
        def contour(x):
            if x_dict["x_inj"] <= x and x <= 0: 
                return chamber_contour(x)
            elif 0 < x <= x_dict["x_tout"]:
                return -np.sqrt(-x**2+r_tout**2) + r_t +r_tout
            elif x_dict["x_tout"] < x and x <= x_dict["x_e"]:
                return np.tan(theta_e)*(x-r_tout*np.sin(theta_e))+r_t+r_tout*(1-np.cos(theta_e))
            else:
                raise ValueError(f"x coordinate exceeds bounds of nozzle. please make sure {x_dict['x_inj']} < x <= {x_dict['x_e']}")

        # x_coords = np.linspace(x_dict["x_inj"], x_dict["x_e"], fidelity)
        x_coords = np.sort(np.append(np.linspace(x_dict["x_inj"], x_dict["x_e"], fidelity), list(x_dict.values())))
        contour_coords = []
        for x in x_coords: 
            
            contour_coords.append([x, contour(x)])

        print(f"r_t: {r_t}, r_tout: {r_tout}")

        self.contour = contour
        self.x_dict = x_dict
        self.contour_coords = contour_coords

        return contour_coords

    def parabolic_contour():
        return
    
    def bell_contour():
        return

In [41]:
def ranged_sim_rocketcow(oxidizer, fuel, of_arr, p_arr, exit_value, gas, condensate = None, size_value=None, size_parameter ="thrust", exit_parameter="pressure"):
    state_list = []
    start_time = time.time()
    for pressure in p_arr:
        for of_ratio in of_arr:
            state = Engine(oxidizer, fuel, of_ratio, pressure, exit_value, gas, size_value, condensate==condensate,  size_parameter=size_parameter,
                           exit_parameter=exit_parameter)
            t1 = time.time()
            state_list.append(state())
            t2 = time.time() -t1
    iter_time = time.time() - start_time

    states = pd.concat(state_list, keys = list(range(len(state_list))))

    concat_time = time.time()- start_time-iter_time

    print(f"-----TOTAL-----\ntotal time: {time.time()-start_time}\niter time: {iter_time}\nconcat time: {concat_time},\ntime per case: {(time.time()-start_time)/(len(p_arr)*len(of_arr))}\nnumber of cases: {(len(p_arr)*len(of_arr))}")
    return states

In [42]:
def chemistry_initializer(oxidizer, fuel, temp_oxidizer=None, temp_fuel=None, combustion_products=None):
    propellant_array = [['oxidizer', oxidizer, temp_oxidizer],['fuel', fuel, temp_fuel]]
    reactants_all   = {S.name: S for S in ct.Species.list_from_file('chem_prop/reactants.yaml')}
    gaseous_all     = {S.name: S for S in ct.Species.list_from_file('chem_prop/gaseous_products.yaml')}
    condensed_all   = {S.name: S for S in ct.Species.list_from_file('chem_prop/condensed_products.yaml')}
    propellant = {}
    propellant_elements = []

    def transport_species(gaseous_species):
        transport_species = [sp for sp in gaseous_species.values() if sp.transport is not None]
        if not transport_species:
            return None
        transport = ct.Solution(thermo="ideal-gas", transport = "mix", species = transport_species)
        return transport
    
    # initialize propellants
    for type, name, temp in propellant_array:
        if name in reactants_all:
            if temp is None:
                if reactants_all[name].thermo.input_data['model'] == 'constant-cp':
                    temp = reactants_all[name].thermo.input_data['T0']
                else:
                    temp= 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[reactants_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')
        
        elif name in gaseous_all:
            if temp is None:
                temp = 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[gaseous_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')

        
        elif name in condensed_all:
            if temp is None:
                temp = 298.15
            propellant[type] = ct.Solution(thermo='ideal-gas', species=[condensed_all[name]])
            propellant[type].TP = temp, 2e5
            propellant[type].equilibrate('TP')

        else:
            raise ValueError(f"{name} does not exist in the thermal database.")    
        
        propellant_elements.extend(propellant[type].element_names)

    # ensures propellant elements are unique
    propellant_elements = list(set(propellant_elements))

    if combustion_products == None: 
        condensed_species = {k: v for k, v in condensed_all.items() if set(v.composition.keys()).issubset(propellant_elements)}
        gaseous_species = {k: v for k, v in gaseous_all.items() if set(v.composition.keys()).issubset(propellant_elements) and ('+' not in k and '-' not in k)}
        transport = transport_species(gaseous_species)
    else: 
        # checks if designated combustion products exist in the thermal database. if it does then filters the data
        if set(combustion_products).issubset(set(condensed_all.keys()) | set(gaseous_all.keys())):
            condensed_species = {k: v for k, v in condensed_all.items() if k in combustion_products and set(v.composition.keys()).issubset(propellant_elements)}
            gaseous_species = {k: v for k, v in gaseous_all.items() if k in combustion_products and set(v.composition.keys()).issubset(propellant_elements) and ('+' not in k and '-' not in k)}
            transport = transport_species(gaseous_species)
        else:
            invalid_species = set(combustion_products) - (set(condensed_all.keys()) | set(gaseous_all.keys()))
            raise ValueError(f"the following species {invalid_species} do not exist in the thermal database.")   
    
    if condensed_species:
        condensate = ct.Solution(thermo='ideal-gas', species=list(condensed_species.values()))
    else: 
        condensate = None
    if gaseous_species:
        gas =  ct.Solution(thermo='ideal-gas', species=list(gaseous_species.values()))
    else: 
        raise ValueError("No valid gas phase combustion products of given propellants. Consider changing propllants or inputted combustion products")        

    return propellant['oxidizer'], propellant['fuel'], gas, condensate, transport

In [43]:
oxidizer, fuel, gas, condensate, transport = chemistry_initializer('O2(L)', "CH4(L)")



In [44]:
of = 1.5
p = 20*1e5
Ox = 'O2(L)' 
f = 'CH4(L)'
p_atm = 1.01325*1e5


oxidizer, fuel, gas, condensate, transport = chemistry_initializer('O2(L)', "CH4(L)")

                # (oxidizer, fuel, of_ratio, pressure, exit_value, gas, condensate = None, size_value=None, size_parameter ="thrust",  exit_parameter="pressure")  
example = EngineState(oxidizer, fuel, of, p, p_atm, gas, exit_parameter="pressure", transport=transport)

example()

Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.32143497467041016
chemistry initialization: 0.0
chamber properties: 0.014528751373291016
throat time: 0.0
exit time: 0.30489158630371094
sizing time: 0.0
df making: 0.002014636993408203 


,of ratio,pressure,temperature,density,specific volume,enthalpy,internal energy,gibbs,entropy,molar mass,c_p,c_v,gamma,gamma_s,speed sound,velocity,mach,area ratio,I_sp,I_vac,c*,C_f,mole fraction,dpi_dlnT_P,dlnn_dlnT_P,dpi_dlnP_T,dlnn_dlnP_T,dlnV_dlnT_P,dlnV_dlnP_T
chamber,1.5,2.000000e+06,1668.337266,1.927922,0.518693,-2.468217e+06,-3.505603e+06,-2.759732e+07,15062.365423,13.371421,2820.505657,2195.127744,1.284894,1.284435,1154.320111,0.000000,0.000000,NaN,NaN,NaN,1532.723141,NaN,"{'CH4': 8.783943068289534e-05, 'CO': 0.3078037...","[-11.859071768744014, -1.5426121518042735, 16....",0.003045,"[0.9977358655039467, 0.4997097739930175, 0.001...",-0.000357,1.003045,-1.000357
throat,1.5,1.096336e+06,1460.104142,1.208022,0.827800,-3.052008e+06,-3.959555e+06,-2.504463e+07,15062.365423,13.376709,2836.893744,2202.609945,1.287969,1.286524,1080.546107,1080.546107,1.000000,NaN,NaN,NaN,NaN,NaN,"{'CH4': 0.00028210533737587046, 'CO': 0.301226...","[-13.235202700257386, -1.4871192553355295, 19....",0.010749,"[0.99270279178388, 0.49909109482678843, 0.0061...",-0.001123,1.010749,-1.001123
exit,1.5,1.013250e+05,979.484078,0.173624,5.759570,-4.765652e+06,-5.349240e+06,-1.951900e+07,15062.365423,13.954840,10472.808248,8510.552185,1.230567,1.161571,823.334362,2143.564699,2.603517,3.50729,218.582768,246.344683,NaN,1.398534,"{'CH4': 0.02190347025314723, 'CO': 0.243998891...","[-13.064034758064158, -0.7551819697471438, 26....",0.867897,"[0.5865226138476243, 0.4571906053044208, 0.305...",-0.059399,1.867897,-1.059399


In [45]:
of_arr = np.arange(1, 5, .5)
p_arr = np.arange(Q(400, 'psi').to_base_units().magnitude, Q(1001, 'psi').to_base_units().magnitude, Q(100, 'psi').to_base_units().magnitude)
exit_value = 8 #in^2
rocketcow_results = ranged_sim_rocketcow(oxidizer, fuel, of_arr, p_arr, exit_value, gas, size_value=None, size_parameter ="thrust", exit_parameter="area ratio")

rocketcow_results

Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.036298513412475586
chemistry initialization: 0.0
chamber properties: 0.022337913513183594
throat time: 0.0054132938385009766
exit time: 0.008547306060791016
sizing time: 0.0
df making: 0.0 
Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.01720261573791504
chemistry initialization: 0.0
chamber properties: 0.005743265151977539
throat time: 0.0022695064544677734
exit time: 0.005578756332397461
sizing time: 0.0
df making: 0.0036110877990722656 
Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.017222166061401367
chemistry initialization: 0.0
chamber properties: 0.0
throat time: 0.009392261505126953
exit time: 0.007829904556274414
sizing time: 0.0
df making: 0.0 
Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.03678631782531738
chemistry initialization: 0.0
chamber properties: 0.018979787826538086
throat time: 0.008399248123168945
exit time: 0.00724077224

of ratio      pressure  temperature   density  specific volume  \
0  chamber       1.0  2.757903e+06  1201.340571  3.642337         0.274549   
   throat        1.0  1.576599e+06  1130.996682  2.258462         0.442779   
   exit          1.0  5.468140e+04   842.301302  0.118394         8.446401   
1  chamber       1.5  2.757903e+06  1668.830741  2.658146         0.376202   
   throat        1.5  1.512989e+06  1461.813803  1.665974         0.600250   
...              ...           ...          ...       ...              ...   
54 throat        4.0  3.998502e+06  3392.311330  3.317146         0.301464   
   exit          4.0  1.452079e+05  2594.131523  0.169475         5.900585   
55 chamber       4.5  6.894757e+06  3523.343669  5.657392         0.176760   
   throat        4.5  3.998583e+06  3360.573963  3.485619         0.286893   
   exit          4.5  1.445613e+05  2550.629554  0.178062         5.616029   

                enthalpy  internal energy         gibbs       entropy  \
0  chamber -2.983867e+06    -3.741046e+06 -2.055150e+07  14623.356901   
   throat  -3.390384e+06    -4.088469e+06 -1.992935e+07  14623.356901   
   exit    -5.295229e+06    -5.757090e+06 -1.761250e+07  14623.356901   
1  chamber -2.468217e+06    -3.505746e+06 -2.727134e+07  14862.578004   
   throat  -3.051449e+06    -3.959620e+06 -2.477777e+07  14862.578004   
...                  ...              ...           ...           ...   
54 throat  -2.113885e+06    -3.319289e+06 -4.161563e+07  11644.493109   
   exit    -5.489298e+06    -6.346110e+06 -3.569664e+07  11644.493109   
55 chamber -1.343164e+06    -2.561880e+06 -4.103348e+07  11264.957061   
   throat  -1.987374e+06    -3.134540e+06 -3.984410e+07  11264.957061   
   exit    -5.200336e+06    -6.012197e+06 -3.393307e+07  11264.957061   

            molar mass           c_p           c_v     gamma   gamma_s  \
0  chamber   13.191722  13433.602749  10224.086093  1.313917  1.175595   
   throat    13.470606  14968.097569  11462.035615  1.305885  1.164664   
   exit      15.163182  21395.194100  17093.340754  1.251668  1.121663   
1  chamber   13.373538   2833.422948   2205.161546  1.284905  1.284047   
   throat    13.383156   2890.699441   2245.955946  1.287069  1.284409   
...                ...           ...           ...       ...       ...   
54 throat    23.398997   7345.960016   6285.635756  1.168690  1.123228   
   exit      25.173331   6169.796674   5455.109242  1.131012  1.109482   
55 chamber   24.037346   6871.228386   5857.041508  1.173157  1.126789   
   throat    24.356866   6829.480542   5861.101625  1.165221  1.123141   
   exit      26.121622   5183.641172   4585.324192  1.130485  1.113912   

            speed sound     velocity      mach  area ratio        I_sp  \
0  chamber   943.470305     0.000000  0.000000         NaN         NaN   
   throat    901.684249   901.684249  1.000000         NaN         NaN   
   exit      719.758436  2150.052323  2.987186         8.0  219.244321   
1  chamber  1154.225120     0.000000  0.000000         NaN         NaN   
   throat   1080.029219  1080.029219  1.000000         NaN         NaN   
...                 ...          ...       ...         ...         ...   
54 throat   1163.590902  1163.590902  1.000000         NaN         NaN   
   exit      974.996247  2846.885941  2.919894         8.0  290.301575   
55 chamber  1171.851453     0.000000  0.000000         NaN         NaN   
   throat   1135.090001  1135.090001  1.000000         NaN         NaN   
   exit      950.968627  2777.470969  2.920676         8.0  283.223218   

                 I_vac           c*       C_f  \
0  chamber         NaN  1299.201823       NaN   
   throat          NaN          NaN       NaN   
   exit     241.149238          NaN  1.654902   
1  chamber         NaN  1532.823722       NaN   
   throat          NaN          NaN       NaN   
...                ...          ...       ...   
54 throat          NaN          NaN       NaN   
   exit     320.991423      

In [46]:
# contour_list = example.conical_contour(0.01, 4, length_parameter="chamber length", fidelity=10000)
# contour_df = pd.DataFrame(contour_list, columns=["x", "y"])
# px.line(contour_df, x="x", y="y",  range_y=[0,.07])

In [47]:
def CEA_wrap_ranged_sim(oxidizer, fuel, of_arr, p_arr, p_e=1.01325, assumption = 'Equilibrium'):

    f = CEA.Fuel(fuel, sum(CEA.ThermoInterface[fuel].temp_ranges[0])/2)
    o = CEA.Oxidizer(oxidizer, sum(CEA.ThermoInterface[oxidizer].temp_ranges[0])/2)
    problem = CEA.RocketProblem(materials=[o,f], pressure_units='bar')
    collector = CEA.DataCollector(  "o_f", "p", "t_p", "c_p", "t", "t_t", "c_t", "h", "t_h", "c_h", "rho", "t_rho", "c_rho", "son", "t_son", "c_son", "visc", "t_visc", "c_visc", "cond", "t_cond", "c_cond", 
                                    "pran", "t_pran", "c_pran", "mw", "t_mw", "c_mw", "m", "t_m", "c_m", "condensed", "t_condensed", "c_condensed", "cp", "t_cp", "c_cp", "gammas", "t_gammas", "c_gammas",
                                    "gamma", "t_gamma", "c_gamma", "isp", "t_isp", "ivac", "t_ivac", "cf", "t_cf", "cstar", "mach", "phi", "ae", "t_ae", "pip", "t_pip", "prod_c", "prod_t", "prod_e",
                                    "dLV_dLP_t", "t_dLV_dLP_t", "c_dLV_dLP_t", "dLV_dLT_p", "t_dLV_dLT_p", "c_dLV_dLT_p")
    for p in p_arr: 
        problem.set_pressure(p)
        problem.set_pip(p/p_e)
        for of in of_arr: 
            f.wt = 1
            o.wt = of
            problem.set_absolute_o_f()
            collector.add_data(problem.run())
    collector["prod_c"] = list(map(dict, collector['prod_c']))
    collector["prod_t"] = list(map(dict, collector['prod_t']))
    collector["prod_e"] = list(map(dict, collector['prod_e']))
    
    chamber, throat, exit = {}, {}, {}
    chamber['of ratio'], throat['of ratio'], exit['of ratio'] = collector['o_f'], collector['o_f'], collector['o_f']
    chamber['pressure'], throat['pressure'], exit['pressure'] = collector['c_p'], collector['t_p'], collector['p']
    chamber['temperature'], throat['temperature'], exit['temperature'] = collector['c_t'], collector['t_t'], collector['t']
    chamber['density'], throat['density'], exit['density'] = collector['c_rho'], collector['t_rho'], collector['rho']
    chamber['specific volume'], throat['specific volume'], exit['specific volume'] = [1/x for x in collector['c_rho']], [1/x for x in collector['t_rho']], [1/x for x in collector['rho']]
    chamber['enthalpy'], throat['enthalpy'], exit['enthalpy'] = collector['c_h'], collector['t_h'], collector['h']
    chamber['internal energy'], throat['internal energy'], exit['internal energy'] = 0,0,0
    chamber['gibbs'], throat['gibbs'], exit['gibbs'] = 0,0,0
    chamber['entropy'], throat['entropy'], exit['entropy'] = 0,0,0
    chamber['molar mass'], throat['molar mass'], exit['molar mass'] = collector['c_mw'], collector['t_mw'], collector['mw']
    chamber['c_p'], throat['c_p'], exit['c_p'] = collector['c_cp'], collector['t_cp'], collector['cp']
    chamber['c_v'], throat['c_v'], exit['c_v'] = [x/y for x, y in zip(collector['c_cp'], collector['c_gamma'])], [x/y for x, y in zip(collector['t_cp'],collector['t_gamma'])], [x/y for x, y in zip(collector['cp'],collector['gamma'])]
    chamber['gamma'], throat['gamma'], exit['gamma'] = collector['c_gamma'], collector['t_gamma'], collector['gamma']
    chamber['gamma_s'], throat['gamma_s'], exit['gamma_s'] = collector['c_gammas'], collector['t_gammas'], collector['gammas']
    chamber['speed sound'], throat['speed sound'], exit['speed sound'] = collector['c_son'], collector['t_son'], collector['son']
    chamber['velocity'], throat['velocity'], exit['velocity'] = 0, collector['t_son'], [x*y for x, y in zip(collector['mach'],collector['son'])]
    chamber['mach'], throat['mach'], exit['mach'] = 0, 1, collector['mach']
    chamber['area ratio'], throat['area ratio'], exit['area ratio'] = np.nan, collector['t_ae'], collector['ae']
    chamber['I_sp'], throat['I_sp'], exit['I_sp'] = 0, collector['t_isp'], collector['isp']
    chamber['I_vac'], throat['I_vac'], exit['I_vac'] = 0, collector['t_ivac'], collector['ivac']
    chamber['c*'], throat['c*'], exit['c*'] = collector['cstar'], 0, 0
    chamber['C_f'], throat['C_f'], exit['C_f'] = 0, collector['t_cf'], collector['cf']
    chamber['mole fraction'], throat['mole fraction'], exit['mole fraction'] = collector['prod_c'], collector['prod_t'], collector['prod_e']
    chamber['dlnV_dlnP_T'], throat['dlnV_dlnP_T'], exit['dlnV_dlnP_T'] = collector['c_dLV_dLP_t'], collector['t_dLV_dLP_t'], collector['dLV_dLP_t']
    chamber['dlnV_dlnT_P'], throat['dlnV_dlnT_P'], exit['dlnV_dlnT_P'] = collector['c_dLV_dLT_p'], collector['t_dLV_dLT_p'], collector['dLV_dLT_p']

    stacked = pd.concat([pd.DataFrame(chamber), pd.DataFrame(throat), pd.DataFrame(exit)], keys=['chamber', 'throat', 'exit'])
    df = stacked.swaplevel(0, 1).sort_index(level=0, sort_remaining=False)
    
    df['enthalpy'] = df['enthalpy'] * 1e3
    df['c_p'] = df['c_p'] * 1e3
    df['c_v'] = df['c_v'] * 1e3


    return df

In [48]:
of_arr = np.arange(1, 4.01, .5).tolist()
p_arr = np.arange(20, 40, 5).tolist()
Ox = 'O2(L)' 
f = 'CH4(L)'
p_atm = 1.01325

oxidizer, fuel, gas, condensate, transport = chemistry_initializer(Ox, f)

rocketcow_results = ranged_sim_rocketcow(oxidizer, fuel, of_arr, (np.array(p_arr)*1e5).tolist(), p_atm*1e5, gas).drop(columns=['dpi_dlnT_P', 'dlnn_dlnT_P', 'dpi_dlnP_T', 'dlnn_dlnP_T'])
CEA_results = CEA_wrap_ranged_sim(Ox, f, of_arr, p_arr, p_atm)
rocketcow_composition = rocketcow_results.pop('mole fraction')
CEA_composition = CEA_results.pop('mole fraction')

Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.02922964096069336
chemistry initialization: 0.0
chamber properties: 0.02008366584777832
throat time: 0.0
exit time: 0.009145975112915039
sizing time: 0.0
df making: 0.0 
Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.020847082138061523
chemistry initialization: 0.0
chamber properties: 0.008787870407104492
throat time: 0.0064852237701416016
exit time: 0.003772735595703125
sizing time: 0.0
df making: 0.0018012523651123047 
Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.008089303970336914
chemistry initialization: 0.0
chamber properties: 0.005383968353271484
throat time: 0.0027053356170654297
exit time: 0.0
sizing time: 0.0
df making: 0.0 
Trying VCS equilibrium solver
VCS solver succeeded
-----
total time: 0.01811075210571289
chemistry initialization: 0.0
chamber properties: 0.014233589172363281
throat time: 0.0038771629333496094
exit time: 0.0
sizing time: 0.0
df making

In [49]:
delta = np.divide(np.subtract(rocketcow_results, CEA_results),CEA_results)*100
compare = pd.concat({'Rocket Cow': rocketcow_results, 'CEA': CEA_results, "delta %": delta}, axis=1)

compare.sort_index(axis=1, level=1)

CEA Rocket Cow   delta %         CEA  Rocket Cow   delta %  \
                C_f        C_f       C_f        I_sp        I_sp      I_sp   
0  chamber  0.00000        NaN       NaN    0.000000         NaN       NaN   
   throat   0.65998        NaN       NaN   91.309888         NaN       NaN   
   exit     1.42600   1.482241  3.943993  197.288481  195.397773 -0.958347   
1  chamber  0.00000        NaN       NaN    0.000000         NaN       NaN   
   throat   0.70521        NaN       NaN  110.142712         NaN       NaN   
...             ...        ...       ...         ...         ...       ...   
26 throat   0.65131        NaN       NaN  120.682977         NaN       NaN   
   exit     1.53730   1.562635  1.648045  284.852192  284.907626  0.019461   
27 chamber  0.00000        NaN       NaN    0.000000         NaN       NaN   
   throat   0.65043        NaN       NaN  117.543323         NaN       NaN   
   exit     1.53970   1.566770  1.758155  278.256881  278.311770  0.019726   

                   CEA  Rocket Cow   delta %        CEA Rocket Cow    delta %  \
                 I_vac       I_vac     I_vac area ratio area ratio area ratio   
0  chamber    0.000000         NaN       NaN        NaN        NaN        NaN   
   throat   170.876656         NaN       NaN     1.0000        NaN        NaN   
   exit     224.954128  222.473651 -1.102659     3.9469   3.891193  -1.411422   
1  chamber    0.000000         NaN       NaN        NaN        NaN        NaN   
   throat   195.759429         NaN       NaN     1.0000        NaN        NaN   
...                ...         ...       ...        ...        ...        ...   
26 throat   228.144750         NaN       NaN     1.0000        NaN        NaN   
   exit     318.022426  318.091362  0.021676     6.1841   6.185118   0.016460   
27 chamber    0.000000         NaN       NaN        NaN        NaN        NaN   
   throat   222.446483         NaN       NaN     1.0000        NaN        NaN   
   exit     310.978593  311.043632  0.020914     6.2546   6.255241   0.010246   

               CEA   Rocket Cow   delta %      CEA    Rocket Cow    delta %  \
                c*           c*        c*      c_p           c_p        c_p   
0  chamber  1357.2  1292.770293 -4.747252  14608.0  13824.513671  -5.363406   
   throat      0.0          NaN       NaN  16175.0  15447.332912  -4.498715   
   exit        0.0          NaN       NaN  18829.0  21147.836015  12.315237   
1  chamber  1532.2  1532.723141  0.034143   2820.9   2820.505657  -0.013979   
   throat      0.0          NaN       NaN   2838.7   2836.893744  -0.063630   
...            ...          ...       ...      ...           ...        ...   
26 throat      0.0          NaN       NaN   7830.5   7844.948557   0.184516   
   exit        0.0          NaN       NaN   5291.9   5307.184072   0.288820   
27 chamber  1772.8  1741.995042 -1.737644   7983.3   7996.401509   0.164111   
   throat      0.0          NaN       NaN   7979.5   7992.579680   0.163916   
   exit        0.0          NaN       NaN   6945.2   6956.894152   0.168377   

                     CEA    Rocket Cow    delta %      CEA Rocket Cow  \
                     c_v           c_v        c_v  density    density   
0  chamber  11312.136034  10554.236453  -6.699880  2.65870   2.673540   
   throat   12607.147572  11861.257713  -5.916405  1.64510   1.657426   
   exit     15130.248145  16701.245815  10.383159  0.19290   0.199147   
1  chamber   2195.466095   2195.127744  -0.015411  1.92800   1.927922   
   throat    2203.994749   2202.609945  -0.062832  1.20810   1.208022   
...                  ...           ...        ...      ...        ...   
26 throat    6706.782542   6718.982060   0.181898  1.62650   1.627018   
   exit      4662.119138   4676.002436   0.297789  0.11143   0.111441   
27 chamber   6782.063526   6793.079641   0.162430  2.77950   2.780543   
   throat    6825.751106   6837.113773   0.166468  1.71210   1.712697   
   exit      6130.336005   6140.533381   0.

In [50]:
CEA_composition[1]['chamber']

{'CH4': 9e-05,
 'CO': 0.3078,
 'CO2': 0.0255,
 'H': 2e-05,
 'H2': 0.52395,
 'H2O': 0.14264}

In [51]:
rocketcow_composition[1]['chamber']

{'CH4': 8.783943062747398e-05,
 'CO': 0.30780374071837463,
 'CO2': 0.02549599474181703,
 'H': 1.775566873995062e-05,
 'H2': 0.5239310951870976,
 'H2O': 0.1426613607219193}

In [52]:
# def pypropep_to_dataframe(p, ox, fuel):

#     Parameters = ['of (wt ratio)', 'p (psi)', 't (K)', 'rho (kg/m^3)', 'v (m/s)', 'Isp (s)', 'Ivac (m/s)', 'c* (m/s)', 'cf', 'sound (m/s)', 'A/At', 'cp (kJ/kg-K)', 'cv (kJ/kg-K)', 'gamma', 'mol mass (g/mol)', 
#               'h (kJ/kg)', 'u (kJ/kg)', 'g (kJ/kg)', 's (kJ/kg-K)', 'dV_P', 'dV_T', 'composition']
#     positions = ['chamber', 'throat', 'exit']

#     df = pd.DataFrame(columns=Parameters, index=positions, dtype=float)

#     df.attrs = {'ox' : [ox.formula(), ox['name'], ox['id']], 
#                 'fuel' : [fuel.formula(), fuel['name'], fuel['id']]}

#     for i, c in enumerate(positions):
#         composition = p.composition[c][0:8]
#         if(bool(p.composition_condensed[c])):
#             composition.append(p.composition_condensed[c])

#         df.loc[c, 'of (wt ratio)']      = p._equil_structs[0].propellant.coef[1] * ox.mw
#         df.loc[c, 'p (atm)']            = p._equil_objs[i].properties.P
#         df.loc[c, 't (K)']              = p._equil_objs[i].properties.T
#         df.loc[c, 'rho (kg/m^3)']       = (p._equil_objs[i].properties.P * 101325 * p._equil_objs[i].properties.M / 1000) / (p._equil_objs[i].properties.T * 8.314 ) # rho (kg/m^3) = (P (atm) * 101325 (Pa) / 1 (atm) * M (g/mol) * 1 kg/1000 g)/( T (K) * R (m^3-Pa/mol-k))
#         df.loc[c, 'v (m/s)']            = p._equil_structs[i].performance.Isp
#         df.loc[c, 'Isp (s)']            = p._equil_structs[i].performance.Isp/sp.constants.g
#         df.loc[c, 'Ivac (m/s)']         = p._equil_structs[i].performance.Ivac
#         df.loc[c, 'c* (m/s)']           = p._equil_structs[i].performance.cstar
#         df.loc[c, 'cf']                 = p._equil_structs[i].performance.cf
#         df.loc[c, 'sound (m/s)']        = p._equil_structs[i].performance.ae_at
#         df.loc[c, 'A/At']               = p._equil_objs[i].properties.Vson
#         df.loc[c, 'cp (kJ/kg-K)']       = p._equil_objs[i].properties.Cp
#         df.loc[c, 'cv (kJ/kg-K)']       = p._equil_objs[i].properties.Cv
#         df.loc[c, 'gamma']              = p._equil_objs[i].properties.Isex
#         df.loc[c, 'mol mass (g/mol)']   = p._equil_objs[i].properties.M
#         df.loc[c, 'h (kJ/kg)']          = p._equil_objs[i].properties.H
#         df.loc[c, 'u (kJ/kg)']          = p._equil_objs[i].properties.U
#         df.loc[c, 'g (kJ/kg)']          = p._equil_objs[i].properties.G
#         df.loc[c, 's (kJ/kg-K)']        = p._equil_objs[i].properties.S
#         df.loc[c, 'dV_P']               = p._equil_objs[i].properties.dV_P
#         df.loc[c, 'dV_T']               = p._equil_objs[i].properties.dV_T
#         df.loc[c, 'composition']        = p.composition[c]
#         # comp_dict = {}
#         # for k, v in p.composition:
#         #     comp_dict[k] = v
#         # print(comp_dict)

#     return df

# o = ppp.PROPELLANTS['OXYGEN (LIQUID)']
# f = ppp.PROPELLANTS['METHANE']

# assumption = 'SHIFTING'
# performance = ppp.ShiftingPerformance()
# performance.add_propellants_by_mass([(f, 1.0), (o, 2)])
# performance.set_state(P=40, Pe=1)
# # df = pypropep_to_dataframe(performance, o, f)
# p = performance
# p.composition['chamber']
# for k, v in p.composition['chamber']:
#     print(k, v)

# def ranged_sim_pypropep(ox, fuel, of_arr, p_arr , p_e = 1, assumption = 'SHIFTING'):

#     # iterates through OF ratios and pressures. 
#     df_list = [] 
#     o = ppp.PROPELLANTS[ox]
#     f = ppp.PROPELLANTS[fuel]

#     if assumption == 'SHIFTING':
#         for p in p_arr:
#             for of in of_arr:
#                 # print(p, of)
#                 performance = ppp.ShiftingPerformance()
#                 performance.add_propellants_by_mass([(f, 1.0), (o, of)])
#                 performance.set_state(P=p, Pe=p_e)
#                 df = pypropep_to_dataframe(performance, o, f)
#                 df_list.append(df)
        
#     elif assumption == 'FROZEN':
#         for p in p_arr:
#             for of in of_arr:
#                 # # print(p, of)
#                 # print(f)
#                 performance = ppp.FrozenPerformance()
#                 performance.add_propellants_by_mass([(f, 1.0), (o, of)])
#                 performance.set_state(P = p, Pe = p_e)
#                 # print(performance)
#                 df = pypropep_to_dataframe(performance, o, f)
#                 df_list.append(df)    
                  
#     else: 
#         raise Exception('invalid assumption, opt ions are \'SHIFTING\' or \'FROZEN\'')
        
#     results = pd.concat(df_list, keys = list(range(len(df_list)))) 
    
#     # if not a list of dataframes is output
#     return results

# of_arr = np.arange(1, 5, 1)
# p_arr = np.arange(400/14.696, 1001/14.696, 100/14.696)
# pypropep_results = ranged_sim_pypropep('OXYGEN (LIQUID)', 'METHANE', of_arr, p_arr, assumption='FROZEN')
# pypropep_results